# 1导入库函数

In [17]:
import tensorflow as tf
print("TF version:",tf.__version__)
#检测TensorFlow是否支持GPU
print("GPU is","available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

TF version: 2.0.0
GPU is NOT AVAILABLE


In [18]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time
import glob
 
from IPython import display

# 导入准备好的数据

    #### 使用Fashion MNIST进行GAN训练，生成器将生成类似于FashionMNIST的数据集 

In [19]:
#导入fashion-mnist数据集
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()


print(train_images.shape)

(60000, 28, 28)


In [20]:
train_images = train_images.reshape(train_images.shape[0],28,28,1).astype('float32')
train_images = (train_images - 127.5)/127.5  #将图片标准化到[-1,1]区间内

print(train_images.shape)
#将标签进行独热编码
train_labels = tf.one_hot(train_labels,depth = 10)
train_labels = tf.cast(train_labels,tf.float32)


(60000, 28, 28, 1)


In [21]:
buffer_size = 60000
batch_size = 256

In [22]:
#批量化和打乱数据
train_dataset = tf.data.Dataset.from_tensor_slices((train_images,train_labels)).shuffle(buffer_size).batch(batch_size)

# 2 构建模型

## 2.1 定义生成器 

In [23]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(7*7*256,use_bias = False,input_shape = (110,))) #因为加入了标签信息，input_shape发送改变
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    
    model.add(layers.Reshape((7,7,256)))
    assert model.output_shape == (None,7,7,256)  #注意： batch size未限制
    
    model.add(layers.Conv2DTranspose(128,(5,5),strides = (1,1),padding = 'same',use_bias = False))
    assert model.output_shape == (None,7,7,128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    

    model.add(layers.Conv2DTranspose(64,(5,5),strides = (2,2),padding = 'same',use_bias = False))
    assert model.output_shape == (None,14,14,64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Conv2DTranspose(1,(5,5),strides = (2,2),padding = 'same',use_bias = False,activation = 'tanh'))
    assert model.output_shape == (None,28,28,1)
    
    return model

    #### 查看生成器网络结构

In [24]:
generator = make_generator_model()
generator.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12544)             1379840   
_________________________________________________________________
batch_normalization (BatchNo (None, 12544)             50176     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 12544)             0         
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 256)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 7, 7, 128)         819200    
_________________________________________________________________
batch_normalization_1 (Batch (None, 7, 7, 128)         512       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 7, 7, 128)         0

## 2.2定义判别器

    #### 判别器可以视作一个CNN分类

In [25]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64,(5,5),strides = (2,2),padding = 'same',input_shape = [28,28,11])) # input_shape加入了标签信息   
    
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv2D(128,(5,5),strides = (2,2),padding = 'same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(1))
   
    return model

    #### 查看判别器网络结构

In [26]:
discriminator = make_discriminator_model()
discriminator.summary()



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 14, 14, 64)        17664     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 128)         204928    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 7, 7, 128)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)             

## 2.3 定义损失函数和优化器

In [27]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits = True)

    #### 判别器损失函数

In [28]:
def discriminator_loss(real_output,fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output),real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output),fake_output)
    total_loss = real_loss+fake_loss
    return total_loss

    #### 生成器损失函数

In [29]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output),fake_output)

    #### 两者的优化器

In [30]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

## 2.4 保存检查点

In [31]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir,"ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer = generator_optimizer,discriminator_optimizer = discriminator_optimizer,generator = generator,discriminator = discriminator)

# 3 训练模型

## 3.1 定义超参数

    #### 定义超参数 

In [32]:
epochs = 100
noise_dim = 100
num_examples_to_generate = 100


    ####产生随机种子作为输入 

In [33]:
#后面将重复使用该种子 （因此在动画GIF中更容易可视化进度
seed = tf.random.normal([num_examples_to_generate,noise_dim])
print(seed.shape)

#有规律设置标签
labels = [i%10 for i in range(num_examples_to_generate)] #舒适化标签向量

labels = tf.one_hot(labels,depth = 10) #将标签独热编码
print(labels.shape)
labels = tf.cast(labels,tf.float32) #转换为tf.float32类型
print(labels.shape)
seed = tf.concat([seed,labels],1) #和图像数据连接起来作为后面的输入数据
print(seed.shape)


(100, 100)
(100, 10)
(100, 10)
(100, 110)


## 3.2 定义训练过程

    #### 训练在生成器接收到一个种子作为输入时开始，用于生产一张图片，判别器随后被用于区分真实图片（自选训练集）和伪造图片（由生成器生成）。 针对这里的每一个模型都计算损失函数，并且计算梯度用于更新生成器与判别器。

In [39]:

#  注意  'tf.function'的使用，该注解使函数被“编译”
@tf.function
def train_step(data_batch):
    
    
    images = data_batch[0] # 图像数据
    labels = data_batch[1] # 标签数据
    print(labels.shape)
    print(images.shape)
    with tf.GradientTape() as gen_tape,tf.GradientTape() as disc_tape:
        noise = tf.random.normal([images.get_shape()[0], noise_dim]) # 产生噪音图像
        print(images.get_shape()[0])

        print(noise.shape)
        noise_input = tf.concat([noise,labels],1)        # 噪音图像数据连接标签数据
        generated_images = generator(noise_input,training = True) # 生成图像
        
        labels_input = tf.reshape(labels,[images.get_shape()[0],1,1,10])
        
        images_input = tf.concat([images,labels_input*tf.ones([images.get_shape()[0],28,28,10])],3)
        # 生成图像数据连接上标签数据
        generated_input = tf.concat([generated_images,labels_input*tf.ones([images.get_shape()[0],28,28,10])],3)
        
        real_output = discriminator(images_input,training = True) # 真实图像的判决结果
        fake_output = discriminator(generated_input,training = True) # 生成图像的判决结果
        
        gen_loss = generator_loss(fake_output)  # 计算生成器损失
        disc_loss = discriminator_loss(real_output,fake_output) # 计算判别器损失
        
    gradients_of_generator = gen_tape.gradient(gen_loss,generator.trainable_variables) # 求梯度
    gradients_of_discriminator = disc_tape.gradient(disc_loss,discriminator.trainable_variables)
    
      # 优化变量
    generator_optimizer.apply_gradients(zip(gradients_of_generator,generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator,discriminator.trainable_variables))






## 3.3 定义训练循环

In [40]:
def train(dataset,epochs):
    for epoch in range(epochs):
        start = time.time()
        
        for data_batch in dataset:
            train_step(data_batch)
        
        #生成图片
        display.clear_output(wait = True)
        generate_and_save_images(generator,epoch+1,seed)
        
        #每5 epochs进行一次存储
        if(epoch+1) % 5 ==  0:
            checkpoint.save(file_prefix = checkpoint_prefix)
            
        print('Time for epoch {} is {} sec'.format(epoch+1,time.time()-start))


## 3.4 生成和保存图片

In [ ]:
def generate_and_save_images(model,epoch,test_input):
    # 注意 'training' 设定值为False 
    # 因此，所有层都在推理模式下运行（batchnorm）。
    
    predictions = model(test_input,training = False)
    
    fig = plt.figure(figsize = (10,10))
    
    for i in range(predictions.shape[0]):
        plt.subplot(10,10,i+1)
        plt.imshow(predictions[i,:,:,0]*127.5+127.5,cmap = 'gray')
        plt.axis('off')
        
    plt.savefig('Cimage_at_epoch_{:04d}.png'.format(epoch))
    plt.show()
   

## 定义训练

    #### 调用上面定义的train（）方法来同时训练生成器和判别器。

    #### 在训练之初，生成的图片看起来像是随机噪声。
    #### 随着训练进行，生成的图片将越来越真实。

In [ ]:

%%time
train(train_dataset,epochs)


(256, 10)
(256, 28, 28, 1)
256
(256, 100)
(256, 10)
(256, 28, 28, 1)
256
(256, 100)
